In [50]:
import csv
import shlex
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import fnmatch
import plotly.io as pio
import scipy
from scipy.signal import find_peaks

In [51]:
df = pd.read_csv('/mnt/bdec1251-cc3f-4208-b17d-816c423896ae/genomics/PyAtBSA/output/A135/A135.complete_table.tsv', sep="\t", header=0)

In [52]:
# def smAlleleFreq(sizeOfBulk, rep):
#     '''
#     An AA/Aa/aa individual carries 0%, 50%, and 100% of the alt (a) allele, respectively.
#     The AA:Aa:aa ratios are 0.25:0.5:0.25 in a F2 population (null hypothesis).
#     '''
    
#     freqL = []
#     pop = [0.0, 0.5, 1.0]
    
#     prob = [0.25, 0.5, 0.25]

#     altFreq = np.random.choice(pop, sizeOfBulk, p=prob).mean()
#     freqL.append(altFreq)

#     return sum(freqL)/len(freqL)


In [53]:
def gStatistic_Array(o1, o3, o2, o4):
    # Calculate G-statistc using numpy array input
    np.seterr(all='ignore')

    e1 = np.where(o1+o2+o3+o4!=0, (o1+o2)*(o1+o3)/(o1+o2+o3+o4), 0)
    e2 = np.where(o1+o2+o3+o4!=0, (o1+o2)*(o2+o4)/(o1+o2+o3+o4), 0)
    e3 = np.where(o1+o2+o3+o4!=0, (o3+o4)*(o1+o3)/(o1+o2+o3+o4), 0)
    e4 = np.where(o1+o2+o3+o4!=0, (o3+o4)*(o2+o4)/(o1+o2+o3+o4), 0)

    llr1 = np.where(o1/e1>0, 2*o1*np.log(o1/e1), 0.0)
    llr2 = np.where(o2/e2>0, 2*o2*np.log(o2/e2), 0.0)
    llr3 = np.where(o3/e3>0, 2*o3*np.log(o3/e3), 0.0)
    llr4 = np.where(o4/e4>0, 2*o4*np.log(o4/e4), 0.0)

    return np.where(e1*e2*e3*e4==0, 0.0, llr1+llr2+llr3+llr4)



In [54]:
#Calculate delta SNP ratio
df['ratio'] = ((df.wt_ref)/(df.wt_ref+df.wt_alt))-((df.mu_ref)/(df.mu_ref+df.mu_alt))

#Drop indels, leave only SNPs
df[
    df["ref"].apply(lambda x: len(x) == 1)
    & df["alt"].apply(lambda x: len(x) == 1)
]

#Calculate ratio_mean for peak identification lower limit
ratio_mean = df["ratio"].mean()

#df.drop(df[df['ratio'] <= 0.1].index, inplace = True) # drop ratios below the defined value. Sometimes necissary

#Drop NAs. Infrequently will arise while div/0 seemingly
df.dropna(axis=0, how='any', subset="ratio", inplace=True)



In [55]:
#lowess smoothing by chromosome
chr_facets=df["chr"].unique()
df_list = []

lowess = sm.nonparametric.lowess
lowess_span=0.3

for i in chr_facets:
    
    # Fit LOWESS ratio ~ position
    df_chr = df[df['chr']==i].copy()

    X=df_chr['pos'].values
    
    Y=df_chr['ratio'].values
    
    y_hat = lowess(Y,X, frac=lowess_span)[:,1]
    
    df_chr['yhat'] = y_hat # fitted values
    
    df_list.append(df_chr)
    
    # Find peaks in the LOWESS fitted curve
    signal = df_chr['yhat'].to_numpy().flatten()
    
    peaks = scipy.signal.find_peaks(
        signal, 
        height=0.5, 
        threshold=None, 
        distance=None, 
        prominence=None, 
        width=None,
        wlen=None, 
        rel_height=None,
        plateau_size=None)
    
    if len(peaks[0]) > 0:
    
        pi = peaks[0] 
        h = peaks[1]['peak_heights']

        # Get index of the tallest peak
        pi_max = (pi[np.argmax(h)]).flatten()

        peak_widths = scipy.signal.peak_widths(
            signal, 
            peaks=pi_max, 
            rel_height=0.5, 
            prominence_data=None, 
            wlen=None)

        peak_bounds = peak_widths[2:4]

        peak_bound_min = np.round(peak_bounds[0])
        peak_bound_max = np.round(peak_bounds[-1])

        peak_bound_min_pos = df_chr['pos'].iloc[peak_bound_min].item()
        peak_bound_max_pos = df_chr['pos'].iloc[peak_bound_max].item()

    # label peaks
        if len(peak_bounds[0]) != 0:
            df_chr['peak'] = [1 if x >= peak_bound_min_pos and x <= peak_bound_max_pos else 0 for x in df_chr['pos']]
    
    else:
        
        df_chr['peak'] = 0
        
df = pd.concat(df_list)

df_peaks = df.loc[df['peak'] == 1]


In [60]:
#Calculate G-statistic to glean more info about potential causal SNPs
df['G_S'] = gStatistic_Array(df['wt_ref'], df['wt_alt'], df['mu_ref'], df['mu_alt'])

#calculate 98th percentile of G-statistics
GS_98p = np.percentile(df['G_S'], 90)

df['G_S_98p'] = [1 if (np.isclose(x, GS_98p) or (x > GS_98p))
                else 0 for x in df['G_S']]

dfLC11=df.loc[(df['peak'] == 1) & (df['G_S_98p'] == 1)]
dfLC01=df.loc[(df['peak'] == 0) & (df['G_S_98p'] == 1)]
dfLC10=df.loc[(df['peak'] == 1) & (df['G_S_98p'] == 0)]

df_likely_cands = pd.concat([dfLC11, dfLC01, dfLC10])

df_likely_cands.to_csv('likely_candidates.tsv', sep='\t')

In [61]:
chr_facets_p=df_peaks["chr"].unique()

#establish fig
fig = px.scatter(
    df,
    x=df['pos'],
    y=df['ratio'],
    facet_col="chr",
    opacity=0.8,
    color_discrete_sequence=['goldenrod'],
    trendline="lowess",
    trendline_options=dict(frac=lowess_span),
    trendline_color_override="blue",
)
            
#format fig
fig.update_traces(
    marker=dict(size=3)
)

fig.update_layout(
    dict(plot_bgcolor = 'white')
)

fig.update_xaxes(
    matches=None
)

fig.for_each_xaxis(
    lambda xaxis: xaxis.update(showticklabels=True)
)

fig.for_each_xaxis(
    lambda xaxis: fig.update_xaxes(tickangle=0)
)

fig.for_each_xaxis(
    lambda x: x.update(title = '')
)

fig.for_each_yaxis(
    lambda y: y.update(title = '')
)


fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=0.5, 
    yref='paper',
    y=-0.1,
    text='Position'
)

fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=-0.8, 
    yanchor='middle',
    yref='paper',
    y=0.6,
    textangle=-90,
    text='Ratio'
)


fig.update_xaxes(
    showgrid=True, 
    gridwidth=0.5, 
    gridcolor='lightgrey'
)

fig.update_xaxes(
    showline=True, 
    linewidth=1, 
    linecolor='black'
)

fig.update_xaxes(
    rangemode="tozero"
)

fig.update_yaxes(
    showgrid=True, 
    gridwidth=0.5, 
    gridcolor='lightgrey'
)

fig.update_yaxes(
    showline=True, 
    linewidth=1, 
    linecolor='black'
)

fig.update_yaxes(
    range=[0, 1]
)

fig.update_layout(
    title=dict(
        text="Linkage map",font=dict(color='black')
    )
)


fig['data'][0]['showlegend'] = True
fig['data'][0]['name'] = 'Polymorphism ratio'

max_pos=df_peaks['pos'].max()
min_pos=df_peaks['pos'].min()

fig.add_trace(
    go.Scatter(
        x=[2],
        y=[2],
        mode='lines',
        name='Lowess Fitted Ratio',
        line=dict(color="blue")))

fig.add_trace(
    go.Scatter(
        x=[0,0], 
        y=[0,0], 
        mode='lines', 
        line=dict(color='black', width=1, dash='dot'),
        name='Identified Peak',
    )
)

#add g-stat > 98 percentile markers
for i in chr_facets:
    df_chr = df[df['chr']==i].copy()
    outliers = df_chr.loc[df_chr['G_S_98p'] == 1 , ["gene","pos","ratio"]]
    fig.add_trace(
    go.Scatter(
        x=outliers['pos'],
        y=outliers['ratio'],
        mode="markers",
        name="G-stat > 98 percentile",
        line=go.scatter.Line(color="red"),
        marker=dict(size=4),
        showlegend=False), row=1, col=i
    )
    
#add peak markers
for i in chr_facets_p:
    df_peaks_chr=df_peaks[df_peaks['chr']==i]
    df_chr=df[df['chr'] == i]
    max_pos=df_peaks_chr['pos'].max()
    min_pos=df_peaks_chr['pos'].min()

    if max_pos > 0:
        max_yhat_pos=df_peaks_chr.loc[df_peaks_chr['yhat'] == df_peaks_chr['yhat'].max()]['pos'].values[0]
        fig.add_vrect(
            x0=min_pos, 
            x1=max_pos, 
            col=i,
            fillcolor="red", 
            opacity=0.2, 
            line_width=0
        )
        
        fig.add_vline(
            x=max_yhat_pos, 
            line_dash="dot", 
            col=i, 
            line_width=1
       )

fig.show()

#pio.write_image(fig, "./plotname.png", scale=6, width=1080, height=540)
#df_peaks.to_csv('peaks_file_name.tsv', sep='\t')


In [62]:
df['T_S'] = df['G_S']*df['ratio']

#establish g-stat lowess fig
fig = px.scatter(df, x=df['pos'], y=df['T_S'],
    facet_col="chr",
    opacity=0.8,
    color_discrete_sequence=['goldenrod'],
    trendline="lowess",
    trendline_options=dict(frac=0.3),
    trendline_color_override="blue")

#format g-stat lowess fig
fig.update_traces(
    marker=dict(size=3)
)

fig.update_layout(
    dict(plot_bgcolor = 'white')
)

fig.update_xaxes(
    matches=None
)

fig.for_each_xaxis(
    lambda xaxis: xaxis.update(showticklabels=True)
)

fig.for_each_xaxis(
    lambda xaxis: fig.update_xaxes(tickangle=0)
)

fig.for_each_xaxis(
    lambda x: x.update(title = '')
)

fig.for_each_yaxis(
    lambda y: y.update(title = '')
)


fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=0.5, 
    yref='paper',
    y=-0.1,
    text='Position'
)

fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=-0.8, 
    yanchor='middle',
    yref='paper',
    y=0.6,
    textangle=-90,
    text='Ratio'
)


fig.update_xaxes(
    showgrid=True, 
    gridwidth=0.5, 
    gridcolor='lightgrey'
)

fig.update_xaxes(
    showline=True, 
    linewidth=1, 
    linecolor='black'
)

fig.update_xaxes(
    rangemode="tozero"
)

fig.update_yaxes(
    showgrid=True, 
    gridwidth=0.5, 
    gridcolor='lightgrey'
)

fig.update_yaxes(
    showline=True, 
    linewidth=1, 
    linecolor='black'
)

fig.update_layout(
    title=dict(
        text="Linkage map",font=dict(color='black')
    )
)


fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=0.5, 
    yref='paper',
    y=-0.1,
    text='Position'
)

fig.add_annotation(
    showarrow=False,
    xanchor='center',
    xref='paper', 
    x=-0.8, 
    yanchor='middle',
    yref='paper',
    y=0.6,
    textangle=-90,
    text='Ratio'
)

fig.add_trace(
    go.Scatter(
    x=[1],
    y=[1],
    mode='lines',
    name='Lowess Fitted G-stat',
    line=dict(color="blue")
    )
)


fig['data'][0]['showlegend'] = True
fig['data'][0]['name'] = 'G-statistic'

max_pos=df_peaks['pos'].max()
min_pos=df_peaks['pos'].min()

    
fig.add_trace(go.Scatter(x=[0,0], 
    y=[0,0], 
    mode='lines', 
    line=dict(color='black', width=1, dash='dot'),
    name='Identified Peak',))


pio.write_image(fig, "./plotname.png", scale=6, width=1080, height=540)
df_peaks.to_csv('peaks_file_name.tsv', sep='\t')

fig.show()
